In [1]:
import os

os.chdir('/opt/ml/P-Stage/1-STAGE/')
                                                                                                                                            
import time
import json
import math
import pickle
import random
import warnings
import functools
from datetime import datetime

import wandb
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from coral_pytorch.dataset import proba_to_label, levels_from_labelbatch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from config import get_args
from predict import get_all_datas
from network import ResNetClassification, get_resnet34
from prepare import get_dataloader, get_classes, get_num_classes
from log_helper import log_confusion_matrix_by_images, log_confusion_matrix

from metrics import (
  get_lossfn,
  get_optimizers,
  change_2d_to_1d,
  tensor_to_numpy,
  apply_grad_cam_pp_to_images,
  tensor_images_to_numpy_images,
)

from log_helper import plots_result
  


args = get_args()
args.device = "cuda:0" if torch.cuda.is_available() else "cpu"
args.test = True
args.train_key = "age-coral"
args.loss_metric = "coral_loss"

train_dataloader, valid_dataloader = get_dataloader(args)

In [2]:
images, labels = next(iter(train_dataloader))

In [3]:
labels

tensor([34, 16, 38,  0, 26, 39, 16, 16,  6, 34,  6, 38, 26, 16, 26, 40,  0, 16,
        16,  0,  0, 40, 40,  6, 41,  4, 26,  6, 34, 34,  6, 41, 41, 34, 38, 16,
         6, 40,  4,  4, 26, 38, 38, 40,  0, 40, 34,  4, 41, 39, 38,  4,  0,  4,
         0, 16, 40, 41, 40, 41,  4, 39, 16, 38])

In [4]:
num_class = get_num_classes(args)
model = get_resnet34(args, num_class).to(args.device)

In [5]:
loss_fn = get_lossfn(args)

43


In [6]:
loss_fn

<function coral_pytorch.losses.coral_loss(logits, levels, importance_weights=None, reduction='mean')>

In [17]:
def get_loss(args, loss_fn, outputs, labels):
      return loss_fn(outputs, labels)
    
def get_label_fn(args):
  
    if args.loss_metric == "coral_loss":

        def _get_label_fn(preds):
            probas = torch.sigmoid(preds)
            labels = proba_to_label(probas)
            return labels

    else:

        def _get_label_fn(preds):
            labels = torch.argmax(preds, dim=1)
            return labels

    return _get_label_fn   

In [33]:
model.train()
epoch_loss = 0.0

num_class = get_num_classes(args)

for idx, (images, labels) in enumerate(train_dataloader):
    print(labels)
    
    if args.loss_metric == "coral_loss":
        some_labels = levels_from_labelbatch(labels, num_classes=num_class)

    images, some_labels = images.to(args.device), some_labels.to(args.device)

    with autocast():
        outputs = model(images)
        loss = get_loss(args, loss_fn, outputs, some_labels)                                                                                                                              

    epoch_loss += loss.item()
    break

tensor([16, 26, 38, 38, 38, 40, 34,  6,  0,  0, 41, 26,  1,  6, 41,  1, 41, 16,
        34,  6, 39, 34, 38, 34,  6, 16, 16, 41, 38, 34, 26, 16, 40, 40,  4, 38,
        41,  1,  6, 26,  1,  6,  0, 41, 40, 34,  1, 16,  6, 34, 26,  6, 40, 26,
         6,  6, 40, 40, 39, 40,  0, 39, 16, 39])


In [34]:
get_labels = get_label_fn(args)

In [35]:
outputs.shape, labels.shape

(torch.Size([64, 42]), torch.Size([64]))

In [36]:
outputs

tensor([[ 0.6245,  0.6245,  0.6245,  ...,  0.6245,  0.6245,  0.6245],
        [-0.0706, -0.0706, -0.0706,  ..., -0.0706, -0.0706, -0.0706],
        [ 0.4622,  0.4622,  0.4622,  ...,  0.4622,  0.4622,  0.4622],
        ...,
        [ 1.3086,  1.3086,  1.3086,  ...,  1.3086,  1.3086,  1.3086],
        [ 0.3120,  0.3120,  0.3120,  ...,  0.3120,  0.3120,  0.3120],
        [-0.0087, -0.0087, -0.0087,  ..., -0.0087, -0.0087, -0.0087]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [37]:
preds = get_labels(outputs)

In [38]:
preds

tensor([42,  0, 42, 42, 42, 42, 42,  0, 42, 42, 42,  0, 42,  0, 42, 42, 42, 42,
         0, 42, 42,  0, 42, 42, 42,  0, 42, 42,  0,  0,  0, 42, 42, 42, 42, 42,
        42,  0, 42,  0, 42, 42, 42, 42, 42,  0,  0, 42, 42,  0,  0, 42, 42, 42,
        42, 42, 42, 42,  0, 42, 42, 42, 42,  0], device='cuda:0')

In [39]:
labels

tensor([16, 26, 38, 38, 38, 40, 34,  6,  0,  0, 41, 26,  1,  6, 41,  1, 41, 16,
        34,  6, 39, 34, 38, 34,  6, 16, 16, 41, 38, 34, 26, 16, 40, 40,  4, 38,
        41,  1,  6, 26,  1,  6,  0, 41, 40, 34,  1, 16,  6, 34, 26,  6, 40, 26,
         6,  6, 40, 40, 39, 40,  0, 39, 16, 39])

In [40]:
preds, labels = tensor_to_numpy(preds), tensor_to_numpy(labels)

In [41]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(labels, preds, labels=range(42)).shape

(42, 42)

In [45]:
len(set(labels).union(set(preds)))

12

In [46]:
confusion_matrix??

Signature:
confusion_matrix(
    y_true,
    y_pred,
    *,
    labels=None,
    sample_weight=None,
    normalize=None,
)
Source:   
@_deprecate_positional_args
def confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None,
                     normalize=None):
    """Compute confusion matrix to evaluate the accuracy of a classification.

    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` and
    predicted to be in group :math:`j`.

    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.

    Read more in the :ref:`User Guide <confusion_matrix>`.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth (correct) target values.

    y_pred : array-like of shape (n_samples,)
        Estimated tar